In [1]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p

## All caps

In [16]:
# Fucntion to get All caps count from the tweets
# pass tweets as it is
def FuncAllCapsCount(GivenTweets):
    AllCapsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        for word in tweet:
            if(word!="I" and re.match("^[A-Z]+$",word)):
                AllCapsCount[i]+=1
    return(AllCapsCount)

## Number of Hashtags

In [137]:
# Fucntion to get HashTag count from the tweets
# pass tweets as it is
def FuncHashtagsCount(GivenTweets):    
    HashTagsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=GivenTweets[i].split(" ")
        for word in tweet:
            if(re.match("^#[a-z]",word)):
                HashTagsCount[i]+=1
    return(HashTagsCount)

## Number of Punctuations like ??? ?!!!

In [19]:
# Fucntion to get COntinuous Punctuation (???,??!,!!) Count and check if last token was a ? or !
# pass tweets as it is
def FuncPunctuationsCount(GivenTweets):
    ContPunctuationCount=np.zeros(len(GivenTweets))
    LastTokenPunctuation=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        token=0
        while token<len(tweet):
            if(tweet[token]=="?" or tweet[token]=="!"):
                index=token+1
                while(index< len(tweet) and (tweet[index]=="?" or tweet[index]=="!")):
                    index+=1
                if(index-token>1):
                    ContPunctuationCount[i]+=1
                    token=index
            token+=1
        if(tweet[len(tweet)-1]=="?" or tweet[len(tweet)-1]=="!"):
            LastTokenPunctuation[i]+=1
    return(ContPunctuationCount,LastTokenPunctuation)

## Using the above functions to extract features

In [4]:
data_train = pd.read_csv('sentiment_train.csv')
data_test = pd.read_csv('sentiment_test.csv')

### Getting All caps count and saving it


In [25]:
Train_AllCapsCount=FuncAllCapsCount(data_train['5'])
Test_AllCapsCount=FuncAllCapsCount(data_test['5'])

100%|██████████| 480000/480000 [01:35<00:00, 5049.09it/s]


In [62]:
df_Train_AllCapsCount=pd.DataFrame(data=Train_AllCapsCount).astype('int')
df_Test_AllCapsCount=pd.DataFrame(data=Test_AllCapsCount).astype('int')
df_Train_AllCapsCount.to_csv("df_Train_AllCapsCount.csv",index=False)
df_Test_AllCapsCount.to_csv("df_Test_AllCapsCount.csv",index=False)

### Getting Hashtag count and saving it


In [57]:
Train_FuncHashtagsCount=FuncHashtagsCount(data_train['5'])
Test_FuncHashtagsCount=FuncHashtagsCount(data_test['5'])

100%|██████████| 480000/480000 [00:07<00:00, 68499.16it/s]


In [63]:
df_Train_FuncHashtagsCount=pd.DataFrame(data=Train_FuncHashtagsCount).astype('int')
df_Test_FuncHashtagsCount=pd.DataFrame(data=Test_FuncHashtagsCount).astype('int')
df_Train_FuncHashtagsCount.to_csv("df_Train_FuncHashtagsCount.csv",index=False)
df_Test_FuncHashtagsCount.to_csv("df_Test_FuncHashtagsCount.csv",index=False)

### Getting Continuous Punctuations and last token ?,! presence and saving it


In [36]:
Train_ContPunctuationCount,Train_LastTokenPunctuation=FuncPunctuationsCount(data_train['5'])
Test_ContPunctuationCount,Test_LastTokenPunctuation=FuncPunctuationsCount(data_test['5'])

100%|██████████| 480000/480000 [01:30<00:00, 5304.63it/s]


In [64]:
df_Train_ContPunctuationCount=pd.DataFrame(data=Train_ContPunctuationCount).astype('int')
df_Test_ContPunctuationCount=pd.DataFrame(data=Test_ContPunctuationCount).astype('int')
df_Train_ContPunctuationCount.to_csv("df_Train_ContPunctuationCount.csv",index=False)
df_Test_ContPunctuationCount.to_csv("df_Test_ContPunctuationCount.csv",index=False)

In [65]:
df_Train_LastTokenPunctuation=pd.DataFrame(data=Train_LastTokenPunctuation).astype('int')
df_Test_LastTokenPunctuation=pd.DataFrame(data=Test_LastTokenPunctuation).astype('int')
df_Train_LastTokenPunctuation.to_csv("df_Train_LastTokenPunctuation.csv",index=False)
df_Test_LastTokenPunctuation.to_csv("df_Test_LastTokenPunctuation.csv",index=False)

## Function to construct a set of all the possible POS tags for training and testing

In [138]:
# Getting the set of all the possible tags from train and test
# Also calculating the count of each tags in train and test set
def PosDictBuild(TrainTweets,TestTweets):
    mytags=set()
    TrainTags=[]
    TestTags=[]
    TrainTweets=TrainTweets.apply(lambda s:s.lower())
    TestTweets=TestTweets.apply(lambda s:s.lower())
    for i in tqdm(range(len(TrainTweets))):
        TempDic={}
        tweet=nltk.word_tokenize(TrainTweets[i])
        tags=nltk.pos_tag(tweet)
        for i in tags:
            mytags.add(i[1])
            if(i[1] in TempDic):
                TempDic[i[1]]+=1
            else:
                TempDic[i[1]]=1
        TrainTags.append(TempDic)
    for i in tqdm(range(len(TestTweets))):
        TempDic={}
        tweet=nltk.word_tokenize(TestTweets[i])
        tags=nltk.pos_tag(tweet)
        for i in tags:
            mytags.add(i[1])
            if(i[1] in TempDic):
                TempDic[i[1]]+=1
            else:
                TempDic[i[1]]=1
        TestTags.append(TempDic)
    return(mytags,TrainTags,TestTags)

In [112]:
mytags,TrainTags,TestTags=PosDictBuild(data_train['5'],data_test['5'])

100%|██████████| 480000/480000 [07:47<00:00, 1026.14it/s]


In [86]:
import pickle
file1=open("TrainTags",'wb')
pickle.dump(TrainTags,file1)
file1.close()

In [87]:
import pickle
file2=open("TestTags",'wb')
pickle.dump(TestTags,file2)
file2.close()

In [97]:
TrainTagsArray = np.zeros((len(TrainTags), len(mytags)), int)
TestTagsArray = np.zeros((len(TestTags), len(mytags)), int)

In [141]:
# Dictionary mapping of tags and integers for the feature vector
counter=0
PosTagDictionary={}
RevPosTagDictionary={}
for i in mytags:
    PosTagDictionary[counter]=i
    RevPosTagDictionary[i]=counter
    counter+=1

## Constructing the POS tag feature

In [108]:
for i in tqdm(range(len(TrainTags))):
    for j in TrainTags[i]:
        TrainTagsArray[i][RevPosTagDictionary[j]]=TrainTags[i][j]    

100%|██████████| 1120000/1120000 [00:05<00:00, 202048.36it/s]


In [113]:
for i in tqdm(range(len(TestTags))):
    for j in TestTags[i]:
        TestTagsArray[i][RevPosTagDictionary[j]]=TestTags[i][j]    

100%|██████████| 480000/480000 [00:02<00:00, 207441.47it/s]


In [124]:
df_Train_Pos_Tags=pd.DataFrame(data=TrainTagsArray)
df_Train_Pos_Tags.rename(columns=PosTagDictionary,inplace=True)

In [125]:
df_Test_Pos_Tags=pd.DataFrame(data=TestTagsArray)
df_Test_Pos_Tags.rename(columns=PosTagDictionary,inplace=True)

In [126]:
df_Train_Pos_Tags.to_csv("df_Train_Pos_Tags.csv",index=False)
df_Test_Pos_Tags.to_csv("df_Test_Pos_Tags.csv",index=False)

##  Functions for Lexicon Features

In [2]:
# Fucnction to construct the BingLiu and MPQA feature and the NRC word emotion association lexicon
# It includes the count of positive and negative words for a particular tweet
# Give tweets after preprocessing
def FuncBingLiu_MPQA_NRCemotion(GivenTweets):
    mpqa_pos=set()
    mpqa_neg=set()
    bing_pos=set()
    bing_neg=set()
    bing_data = pd.read_csv("./lexicons/1. BingLiu.csv", sep='\t', names=["words","sentiment"], header=None)

    for i in range(len(bing_data['words'])):
        if bing_data['sentiment'][i]=='positive':
            bing_pos.add(bing_data['words'][i])
        elif bing_data['sentiment'][i]=='negative':
            bing_neg.add(bing_data['words'][i])
    mpqa_data = pd.read_csv("./lexicons/2. mpqa.txt", sep='\t', names=["words","sentiment"], header=None)

    for i in range(len(mpqa_data['words'])):
        if mpqa_data['sentiment'][i]=='positive':
            mpqa_pos.add(mpqa_data['words'][i])
        elif mpqa_data['sentiment'][i]=='negative':
            mpqa_neg.add(mpqa_data['words'][i])

    polar_pos=mpqa_pos.union(bing_pos)
    polar_neg=mpqa_neg.union(bing_neg)

    emotions_df=pd.read_csv("./lexicons/8. NRC-word-emotion-lexicon.txt", sep='\t', names=['word','emotion','score'], header=None)
    emotions_df.head()
    
    for i in range(len(emotions_df['word'])):
        if emotions_df['emotion'][i]=="positive" and emotions_df['score'][i]==1:
            polar_pos.add(emotions_df['word'][i])

        elif emotions_df['emotion'][i]=="negative" and emotions_df['score'][i]==1:
            polar_neg.add(emotions_df['word'][i])


    polarcount_pos=np.zeros(len(GivenTweets))
    polarcount_neg=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=GivenTweets[i]
        for word in word_tokenize(tweet):
            if word in polar_pos:
                polarcount_pos[i]+=1
            elif word in polar_neg:
                polarcount_neg[i]+=1
    return(polarcount_pos,polarcount_neg)

In [62]:
# Function for Hashtag Sentiment Lexicon
# Pass the tweets as it is
def Func_NRCHashtagSentiment(Giventweets):
    d1 = pd.read_csv("./lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", sep='\t', names=['term','score','numPos','numNeg'], header=None)
    dic_d1={}
    for i in range(len(d1['term'])):
        dic_d1[d1['term'][i]]=d1['score'][i]
    # print(dic_d1)
    aggScoreHashtags=np.zeros(len(Giventweets))
    for i in tqdm(range(len(Giventweets))):
        tweet=Giventweets[i]
        hashtags = [i  for i in tweet.split() if i.startswith("#") ]
        for word in hashtags:
            if word in dic_d1:
                aggScoreHashtags[i]+=dic_d1[word]
            elif word[1:] in dic_d1:
                aggScoreHashtags[i]+=dic_d1[word[1:]]
    return(aggScoreHashtags)

In [54]:
# function for sentiment 140 lexicon
# Give tweets after pre processing
def Func_Sentiment140Lexicon(Giventweets):
    d1 = pd.read_csv("./lexicons/3. Sentiment140-Lexicon-v0.1/unigrams-pmilexicon.txt", sep='\t', names=['term','score','numPos','numNeg'], header=None)
    dic_d1={}
    for i in tqdm(range(len(d1['term']))):
        dic_d1[d1['term'][i]]=d1['score'][i]
    aggScore1=np.zeros(len(Giventweets))
    for i in tqdm(range(len(Giventweets))):
        tweet=Giventweets[i]
        for word in word_tokenize(tweet):
            if word in dic_d1:
                aggScore1[i]+=dic_d1[word]
    return(aggScore1)

# Function to count the elongated words

In [70]:
# Returns the Number of elongated words for each tweet
# (.)\1 represents the repetition of continuous characters
# {2,} the repetition should be more than 2
def elongated_words(Giventweets):
    ElongatedWordCount=np.zeros(len(Giventweets))
    for i in tqdm(range(len(Giventweets))):
        tweet=Giventweets[i]
        for word in word_tokenize(tweet):
            if(len(re.findall(r"(.)\1{2,}",word))>0):
                ElongatedWordCount[i]+=1
    return(ElongatedWordCount)

# Function to count the emoticon score

In [134]:
def emoticon_score(Giventweets):
    emoticons=pd.read_csv("./lexicons/9. AFINN-emoticon-8.txt", sep='\t', names=['emoticon','score'], header=None)
    dic_emoji={}
    for i in tqdm(range(len(emoticons['emoticon']))):
        dic_emoji[emoticons['emoticon'][i]]=emoticons['score'][i]
    PresencePositiveEmoticon=np.zeros(len(Giventweets))
    PresenceNegativeEmoticon=np.zeros(len(Giventweets))
    LastTokenEmoticon=np.zeros(len(Giventweets))
    emoji = re.compile('[\\u203C-\\u3299\\U0001F000-\\U0001F644]')
    emojiScore=np.zeros(len(Giventweets))
    for i in tqdm(range(len(Giventweets))):
        tweet=(Giventweets[i].split(" "))
        for i in range(len(tweet)):
            if(tweet[i] in dic_emoji):
                if(dic_emoji[tweet[i]]>0):
                    PresencePositiveEmoticon[i]+=dic_emoji[tweet[i]]
                elif(dic_emoji[tweet[i]]<0):
                    PresenceNegativeEmoticon[i]+=dic_emoji[tweet[i]]
        if(tweet[len(tweet)-1] in dic_emoji ):
            if(dic_emoji[tweet[len(tweet)-1]]>0):
                LastTokenEmoticon[i]=1
            elif(dic_emoji[tweet[len(tweet)-1]]<0):
                LastTokenEmoticon[i]=0
    return(PresencePositiveEmoticon,PresenceNegativeEmoticon,LastTokenEmoticon)

# Function to count the nagated contexts

In [223]:
# pass tweets as it is and remove punctuations here
def negation_count(GivenTweets):
    # Count of Negating words
    neg_context_count=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.sent_tokenize(GivenTweets[i])
        for j in tweet:
            context=word_tokenize(j)
            context[0]=re.sub(r'[^\w\s]',"",context[0])
            res = re.match(r'never|no|nothing|nowhere|noone|none|not|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint',context[0])
            if(res and re.match(r",|.|:|;|!|\?",context[len(context)-1])):
#                 print(context[0],context[len(context)-1])
                neg_context_count[i]+=1
    return(neg_context_count)


# Using these functions to extract the features and saving them


In [5]:
data_train = pd.read_csv('sentiment_train.csv')
data_test = pd.read_csv('sentiment_test.csv')
tweets_train_original=data_train['5']
tweets_train=data_train['5']
tweets_test_original=data_test['5']
tweets_test=data_test['5']

tweets_train=tweets_train.apply(lambda s:s.lower())
tweets_train_original=tweets_train_original.apply(lambda s:s.lower())

tweets_test=tweets_test.apply(lambda s:s.lower())
tweets_test_original=tweets_test_original.apply(lambda s:s.lower())

In [91]:
for i in tqdm(range(len(tweets_train))):
    #Removing Urls, mentions, hashtags
    res=p.clean(tweets_train[i])
    res = re.sub(r'[^\w\s]', '',res)
    tweets_train[i]=res
for i in tqdm(range(len(tweets_test))):
    #Removing Urls, mentions, hashtags
    res=p.clean(tweets_test[i])
    res = re.sub(r'[^\w\s]', '',res)
    tweets_test[i]=res

100%|██████████| 480000/480000 [00:24<00:00, 19932.03it/s]


In [93]:
train_polarcount_pos,train_polarcount_neg=FuncBingLiu_MPQA_NRCemotion(tweets_train)
test_polarcount_pos,test_polarcount_neg=FuncBingLiu_MPQA_NRCemotion(tweets_test)

100%|██████████| 480000/480000 [00:54<00:00, 8771.04it/s]


In [229]:
df_train_polarcount=pd.DataFrame(train_polarcount_pos)
df_train_polarcount.rename(columns={0:"polarcount_positive"},inplace=True)
df_train_polarcount["polarcount_negative"]=pd.DataFrame(train_polarcount_neg)
df_train_polarcount["polarcount_negative"]=df_train_polarcount["polarcount_negative"].astype(int)
df_train_polarcount["polarcount_positive"]=df_train_polarcount["polarcount_positive"].astype(int)
df_train_polarcount.to_csv("df_train_polarcount.csv",index=False)


df_test_polarcount=pd.DataFrame(test_polarcount_pos)
df_test_polarcount.rename(columns={0:"polarcount_positive"},inplace=True)
df_test_polarcount["polarcount_negative"]=pd.DataFrame(test_polarcount_neg)
df_test_polarcount["polarcount_negative"]=df_test_polarcount["polarcount_negative"].astype(int)
df_test_polarcount["polarcount_positive"]=df_test_polarcount["polarcount_positive"].astype(int)
df_test_polarcount.to_csv("df_test_polarcount.csv",index=False)

In [95]:
train_aggScore=Func_Sentiment140Lexicon(tweets_train)
test_aggScore1=Func_Sentiment140Lexicon(tweets_test)

100%|██████████| 480000/480000 [00:53<00:00, 8986.02it/s]


In [230]:
df_train_aggScore=pd.DataFrame(train_aggScore)
df_train_aggScore.rename(columns={0:"Sentiment140Score"},inplace=True)
df_train_aggScore.to_csv("df_train_aggScore.csv",index=False)

df_test_aggScore=pd.DataFrame(test_aggScore1)
df_test_aggScore.rename(columns={0:"Sentiment140Score"},inplace=True)
df_test_aggScore.to_csv("df_test_aggScore.csv",index=False)


In [97]:
train_HashtagScore=Func_NRCHashtagSentiment(tweets_train_original)
test_HashtagScore=Func_NRCHashtagSentiment(tweets_test_original)

100%|██████████| 480000/480000 [00:02<00:00, 210393.15it/s]


In [231]:
df_train_HashtagScore=pd.DataFrame(train_HashtagScore)
df_train_HashtagScore.rename(columns={0:"NRChashTagscore"},inplace=True)
df_train_HashtagScore.to_csv("df_train_HashtagScore.csv",index=False)

df_test_HashtagScore=pd.DataFrame(test_HashtagScore)
df_test_HashtagScore.rename(columns={0:"NRChashTagscore"},inplace=True)
df_test_HashtagScore.to_csv("df_test_HashtagScore.csv",index=False)

In [99]:
train_ElongatedWordCount=elongated_words(tweets_train_original)
test_ElongatedWordCount=elongated_words(tweets_test_original)

100%|██████████| 480000/480000 [01:28<00:00, 5395.46it/s]


In [232]:
df_train_ElongatedWordCount=pd.DataFrame(train_ElongatedWordCount)
df_train_ElongatedWordCount.rename(columns={0:"ELongatedWordCOunt"},inplace=True)
df_train_ElongatedWordCount["ELongatedWordCOunt"]=df_train_ElongatedWordCount["ELongatedWordCOunt"].astype(int)
df_train_ElongatedWordCount.to_csv("df_train_ElongatedWordCount.csv",index=False)

df_test_ElongatedWordCount=pd.DataFrame(test_ElongatedWordCount)
df_test_ElongatedWordCount.rename(columns={0:"ELongatedWordCOunt"},inplace=True)
df_test_ElongatedWordCount["ELongatedWordCOunt"]=df_test_ElongatedWordCount["ELongatedWordCOunt"].astype(int)
df_test_ElongatedWordCount.to_csv("df_test_ElongatedWordCount.csv",index=False)


In [137]:
Train_PositiveEmoticonScore,Train_NegativeEmoticonScore,Train_LastTokenEmoticon=emoticon_score(tweets_train_original)
Test_PositiveEmoticonScore,Test_NegativeEmoticonScore,Test_LastTokenEmoticon=emoticon_score(tweets_test_original)

100%|██████████| 480000/480000 [00:02<00:00, 193021.12it/s]


In [233]:
df_train_Emoticon=pd.DataFrame(Train_PositiveEmoticonScore)
df_train_Emoticon.rename(columns={0:"PositiveEmoticonScore"},inplace=True)
df_train_Emoticon["NegativeEmoticonScore"]=pd.DataFrame(Train_NegativeEmoticonScore)
df_train_Emoticon["LastTokenEmoticon"]=pd.DataFrame(Train_LastTokenEmoticon)

df_train_Emoticon["PositiveEmoticonScore"]=df_train_Emoticon["PositiveEmoticonScore"].astype(int)
df_train_Emoticon["NegativeEmoticonScore"]=df_train_Emoticon["NegativeEmoticonScore"].astype(int)
df_train_Emoticon["LastTokenEmoticon"]=df_train_Emoticon["LastTokenEmoticon"].astype(int)

df_train_Emoticon.to_csv("df_train_Emoticon.csv",index=False)



df_test_Emoticon=pd.DataFrame(Test_PositiveEmoticonScore)
df_test_Emoticon.rename(columns={0:"PositiveEmoticonScore"},inplace=True)
df_test_Emoticon["NegativeEmoticonScore"]=pd.DataFrame(Test_NegativeEmoticonScore)
df_test_Emoticon["LastTokenEmoticon"]=pd.DataFrame(Test_LastTokenEmoticon)

df_test_Emoticon["PositiveEmoticonScore"]=df_test_Emoticon["PositiveEmoticonScore"].astype(int)
df_test_Emoticon["NegativeEmoticonScore"]=df_test_Emoticon["NegativeEmoticonScore"].astype(int)
df_test_Emoticon["LastTokenEmoticon"]=df_test_Emoticon["LastTokenEmoticon"].astype(int)

# df_test_Emoticon.head()
df_test_Emoticon.to_csv("df_test_Emoticon.csv",index=False)



In [141]:
# df_test_Emoticon.head()

In [224]:
Train_negation_context_count=negation_count(tweets_train_original)
Test_negation_context_count=negation_count(tweets_test_original)

100%|██████████| 480000/480000 [01:49<00:00, 4391.92it/s]


In [234]:
df_Train_negation_context_count=pd.DataFrame(Train_negation_context_count)
df_Train_negation_context_count.rename(columns={0:"negation_context_count"},inplace=True)
df_Train_negation_context_count["negation_context_count"]=df_Train_negation_context_count["negation_context_count"].astype(int)
df_Train_negation_context_count.to_csv("df_Train_negation_context_count.csv",index=False)

df_Test_negation_context_count=pd.DataFrame(Test_negation_context_count)
df_Test_negation_context_count.rename(columns={0:"negation_context_count"},inplace=True)
df_Test_negation_context_count["negation_context_count"]=df_Test_negation_context_count["negation_context_count"].astype(int)
df_Test_negation_context_count.to_csv("df_Test_negation_context_count.csv",index=False)


# Concatenating all features except ngrams ans saving them

In [102]:
Train_Feature=pd.read_csv("./Question1 Features/df_Train_AllCapsCount.csv")
Train_Feature.rename(columns={'0':"AllCapsCount"},inplace=True)
Train_Feature['ContPunctuationCount']=pd.read_csv("./Question1 Features/df_Train_ContPunctuationCount.csv")
Train_Feature['ELongatedWordCount']=pd.read_csv("./Question1 Features/df_train_ElongatedWordCount.csv")
Train_Feature[["PositiveEmoticonScore","NegativeEmoticonScore","LastTokenEmoticon"]]=pd.read_csv("./Question1 Features/df_train_Emoticon.csv")
Train_Feature['HashTagCount']=pd.read_csv("./Question1 Features/df_Train_FuncHashtagsCount.csv")
Train_Feature['NRChashTagscore']=pd.read_csv("./Question1 Features/df_train_HashtagScore.csv")
Train_Feature['LastTokenPunctuation']=pd.read_csv("./Question1 Features/df_Train_LastTokenPunctuation.csv")
Train_Feature['NegationContextCount']=pd.read_csv("./Question1 Features/df_Train_negation_context_count.csv")
Train_Feature[['polarcount_positive', 'polarcount_negative']]=pd.read_csv("./Question1 Features/df_train_polarcount.csv")
Train_Feature[['RP', 'JJ', '\'\'', 'EX', 'VBG', ')', 'NN', 'MD', 'PRP$', 'JJS', 'WP$','IN', 'JJR', ':', 'PDT', 'DT', '(', 'NNPS', ',', 'RBR', '``', 'PRP','WDT', 'VBP', 'VBN', 'SYM', 'WRB', 'RBS', 'VBZ', 'UH', 'CD', 'VB', '#','TO', '$', 'RB', 'NNS', 'FW', 'WP', 'CC', 'POS', 'VBD', '.', 'LS','NNP']]=pd.read_csv("./Question1 Features/df_Train_Pos_Tags.csv")
Train_Feature["Sentiment140Score"]=pd.read_csv("./Question1 Features/df_train_aggScore.csv")

In [104]:
Train_Feature.to_csv("TrainFeatures_without_ngrams",index=False)

In [105]:
Test_Feature=pd.read_csv("./Question1 Features/df_Test_AllCapsCount.csv")
Test_Feature.rename(columns={'0':"AllCapsCount"},inplace=True)
Test_Feature['ContPunctuationCount']=pd.read_csv("./Question1 Features/df_Test_ContPunctuationCount.csv")
Test_Feature['ELongatedWordCount']=pd.read_csv("./Question1 Features/df_test_ElongatedWordCount.csv")
Test_Feature[["PositiveEmoticonScore","NegativeEmoticonScore","LastTokenEmoticon"]]=pd.read_csv("./Question1 Features/df_test_Emoticon.csv")
Test_Feature['HashTagCount']=pd.read_csv("./Question1 Features/df_Test_FuncHashtagsCount.csv")
Test_Feature['NRChashTagscore']=pd.read_csv("./Question1 Features/df_test_HashtagScore.csv")
Test_Feature['LastTokenPunctuation']=pd.read_csv("./Question1 Features/df_Test_LastTokenPunctuation.csv")
Test_Feature['NegationContextCount']=pd.read_csv("./Question1 Features/df_Test_negation_context_count.csv")
Test_Feature[['polarcount_positive', 'polarcount_negative']]=pd.read_csv("./Question1 Features/df_test_polarcount.csv")
Test_Feature[['RP', 'JJ', '\'\'', 'EX', 'VBG', ')', 'NN', 'MD', 'PRP$', 'JJS', 'WP$','IN', 'JJR', ':', 'PDT', 'DT', '(', 'NNPS', ',', 'RBR', '``', 'PRP','WDT', 'VBP', 'VBN', 'SYM', 'WRB', 'RBS', 'VBZ', 'UH', 'CD', 'VB', '#','TO', '$', 'RB', 'NNS', 'FW', 'WP', 'CC', 'POS', 'VBD', '.', 'LS','NNP']]=pd.read_csv("./Question1 Features/df_Test_Pos_Tags.csv")
Test_Feature["Sentiment140Score"]=pd.read_csv("./Question1 Features/df_test_aggScore.csv")

In [108]:
Test_Feature.to_csv("TestFeatures_without_ngrams",index=False)